In [ ]:
import networkx as nx
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from scipy.sparse import coo_matrix
import evaluate

In [ ]:
post = nx.Graph()

In [ ]:
import pickle
with open('/workspace/persistent/ct_df_posts_single_preds.pkl', 'rb') as file:
    # Load the contents of the PKL file
    ndf = pickle.load(file)

In [ ]:
ndf['id']=0
ndf['id']=range(len(ndf))
ndf = ndf.set_index(ndf['id'])

In [ ]:
import pickle
test_indices=[]
with open('/workspace/persistent/test_split_thales.pkl', 'rb') as file:
    # Load the contents of the PKL file
    test_split = pickle.load(file)
test_split=test_split.dropna(subset=['caption'])
test_indices=test_split.index.tolist()
test_split['id']=0
test_split['id']=range(len(test_split))
test_split = test_split.set_index(test_split['id'])
test_target=[]
for i,rows in test_split.iterrows():
    test_target.append(int(test_split['has_disclosures'][i]))

In [ ]:
import pickle
train_indices=[]
with open('/workspace/persistent/train_split_thales.pkl', 'rb') as file:
    # Load the contents of the PKL file
    train_split = pickle.load(file)
train_split=train_split.dropna(subset=['caption'])
train_indices=train_split.index.tolist()
train_split['id']=0
train_split['id']=range(len(train_split))
train_split = train_split.set_index(train_split['id'])
train_target=[]
for i,rows in train_split.iterrows():
    train_target.append(int(train_split['has_disclosures'][i]))

In [ ]:
import pickle
val_indices=[]
with open('/workspace/persistent/eval_split_thales.pkl', 'rb') as file:
    # Load the contents of the PKL file
    eval_split = pickle.load(file)
eval_split=eval_split.dropna(subset=['caption'])
val_indices=eval_split.index.tolist()
eval_split['id']=0
eval_split['id']=range(len(eval_split))
eval_split = eval_split.set_index(eval_split['id'])
eval_target=[]
for i,rows in eval_split.iterrows():
    eval_target.append(int(eval_split['has_disclosures'][i]))

In [ ]:
ndf['size'] = ndf['followers'].apply(lambda x: 1 if x >= 600000 else 2)

In [ ]:
ndf['tagged_users']=ndf['tagged_users'].astype(str)

In [ ]:
# Transforming the 'sponsored' column into numerical values
ndf['has_disclosures'] = np.where(ndf['has_disclosures'], 1, 0)

In [ ]:
category_mapping = {
    'en':1,
    'pt':2,
    'de':3,
    'nl':4,
    'und':5,
    'es':6,
    'fr':7,
    'da':8,
    ' ':9,
    'tr':10,
    'id':11,
    'ar':12
}

ndf['language'] = ndf['language'].map(category_mapping)

In [ ]:
category_mapping = {
    'mega': 1,
    'macro': 2
}

ndf['size'] = ndf['size'].map(category_mapping)

In [ ]:
category_mapping = {
    'BR': 1,
    'US': 2,
    'NL': 3,
    'DE':4
}

ndf['country'] = ndf['country'].map(category_mapping)

In [ ]:
category_mapping = {
    'photo': 1,
    'video': 2,
    'album': 3
}

ndf['type'] = ndf['type'].map(category_mapping)

In [ ]:
for i,rows in ndf.iterrows():
    ndf['date'][i]=ndf['date'][i].timestamp()

In [ ]:
ndf

In [ ]:
count = 0
while count<len(ndf):
    post.add_nodes_from([(ndf['shortcode'][count],{'x':float(ndf['likes'][count]),'type':float(ndf['type'][count]),'country':float(ndf['country'][count]),'size':float(ndf['size'][count]),'language':float(ndf['language'][count]),'followers':float(ndf['followers'][count]),'comments':float(ndf['comments'][count]),'date':float(ndf['date'][count])})])
    count=count+1

In [ ]:
post.nodes['CaSRiPDOpaw']

In [ ]:
brand = nx.Graph()

In [ ]:
branddf = pd.read_csv('/workspace/persistent/brandsdf.csv')
branddf=branddf.dropna(subset=['Category'])
branddf['tag']=branddf['tag'].astype(str)

In [ ]:
branddf['id']=0
branddf['id']=range(len(branddf))
branddf = branddf.set_index(branddf['id'])
branddf['tag'] = branddf['tag'].str[1:]

In [ ]:
pic = []
shortcode = []

for i, rows in ndf.iterrows():
    row_text = re.sub(r"['\[\]]", '', ndf['tagged_users'][i])  # Remove single quotes, square brackets
    row_matches = re.findall(r'[^,]+', row_text)  # Parse after every comma
    pic.extend(row_matches)
    shortcode.extend([ndf['shortcode'][i]] * len(row_matches))

# Create a DataFrame with the matches and associated values
imgdfo = pd.DataFrame({'brand': pic, 'shortcode': shortcode})


In [ ]:
imgdfo

In [ ]:
branddf['size']=None
branddf['size'] = branddf['Followers'].apply(lambda x: 1 if x >= 600000 else 2)

In [ ]:
branddf

In [ ]:
count = 0
while count<len(branddf):
    post.add_nodes_from([(branddf['tag'][count],{'x':0,'type':0,'country':0,'size':float(branddf['size'][count]),'language':0,'followers':float(branddf['Followers'][count]),'comments':0,'date':0})])
    count=count+1

In [ ]:
count

In [ ]:
# Iterate over each row in ndf
for i, row in imgdfo.iterrows():
     post.add_edge(imgdfo['brand'][i], imgdfo['shortcode'][i])

count = 0
for node in post:
    for i in post:
        if post.nodes[node]['tag'] == post.nodes[i]['tag']:
            post.add_edge(node,i)
    count = count+1
    print(count,'/',len(post.nodes()))

In [ ]:
post=nx.convert_node_labels_to_integers(post, first_label=0)

In [ ]:
import numpy as np

def create_mask(indices, size):
    mask = np.zeros(size, dtype=bool)
    mask[indices] = True
    return mask

# Example usage
test_indices = test_indices
train_indices = train_indices
val_indices = val_indices
size = len(post.nodes())

test_mask = create_mask(test_indices, size)
train_mask = create_mask(train_indices, size)
val_mask = create_mask(val_indices, size)

In [ ]:
# Create an empty feature matrix
feature_matrix = np.zeros((len(post.nodes), 7))

# Iterate over the nodes in the 'post' network
for i, node in enumerate(list(post.nodes)[:(len(ndf))]):
    # Get the 'like' and 'comments' attributes of each node
    followers = post.nodes[node]['followers']
    comments = post.nodes[node]['comments']
    like = post.nodes[node]['x']
    tipe = post.nodes[node]['type']
    country = post.nodes[node]['country']
    size = post.nodes[node]['size']
    language = post.nodes[node]['language']
    date = post.nodes[node]['date']
    
    # Assign the feature values to the feature matrix
    feature_matrix[i] = [like, comments,followers,tipe,country,language,date]

# Print the feature matrix
fm=feature_matrix
fm

In [ ]:
import torch
from torch_geometric.data import Data
import networkx as nx
# Extract node features and convert them to a tensor
node_features = torch.tensor(fm, dtype=torch.float)

edge_indices=list(post.edges())
edge_indices = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()

# Optionally, extract edge features if available and convert them to a tensor
edge_features = None  # Replace None with your edge feature extraction logic if needed

truth=[]
truth = [ndf['has_disclosures'][i] for i in range(len(ndf))] + [int(0)] * (len(post)-len(ndf))
truth=torch.tensor(truth)
test_truth =[test_split['has_disclosures'][i] for i in range(len(test_split))]
# Create a PyTorch Geometric Data object
data = Data(x=node_features, edge_index=edge_indices, edge_attr=edge_features,y=truth,train_mask=train_mask,test_mask=test_mask,val_mask=val_mask)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        return F.log_softmax(x, dim=1)

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset (e.g., Cora)


# Move the data to the device
data = data.to(device)

# Create DataLoader for batching
loader = DataLoader([data], batch_size=1, shuffle=True)

# Instantiate the model and move it to the device
model = GCN(input_dim=7, hidden_dim=16, num_classes=2).to(device)

# Set up the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

# Training loop
model.train()
for epoch in range(200):
    for batch in loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

# Evaluate the trained model
model.eval()
pred = model(data.x.to(device), data.edge_index.to(device)).argmax(dim=1).cpu().numpy()

# Convert ground truth labels to numpy array
ground_truth = data.y.cpu().numpy()

# Calculate precision, recall, and F1 score
precision = precision_score(ground_truth, pred)
recall = recall_score(ground_truth, pred)
f1 = f1_score(ground_truth, pred)
accuracy = (pred == ground_truth).mean()

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
# Select the rows corresponding to train_indices using tensor indexing
last_hidden_state = model(data.x, data.edge_index)
selected_rows = last_hidden_state[test_indices]
graph_state=[]
for i in range(len(selected_rows)):
    graph_state.append(torch.tensor(selected_rows[i]))

In [ ]:
import pickle
with open('/workspace/persistent/graph_state_thales.pkl', 'wb') as file:
    pickle.dump(graph_state, file)

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

# Define the self-attention module
class SelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(SelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

    def forward(self, x):
        # Apply self-attention
        output, _ = self.attention(x, x, x)

        return output.squeeze()

# Define the parameters
embedding_dim = 2
num_heads = 2

# Create an instance of the self-attention module
self_attention = SelfAttention(embedding_dim, num_heads).cuda()

# Convert the given tensor to a PyTorch tensor and move to GPU

# Apply self-attention to all elements in the tensor
graph_attention = []
with tqdm(total=len(test_split), desc="Applying Attention") as progress_bar:
    for i,rows in test_split.iterrows():
        # Apply self-attention
        output = (graph_state[i].unsqueeze(0).to('cuda'))
        with torch.no_grad():
            output = output.cpu()
        graph_attention.append(output)
        progress_bar.update(1)

In [ ]:
import pickle
with open('/workspace/persistent/graph_attention_thales.pkl', 'wb') as file:
    pickle.dump(graph_attention, file)

In [ ]:
graph_representation = []
with tqdm(total=len(test_split), desc="Calculating the Image Score") as progress_bar:
    for i,rows in test_split.iterrows():
        xi=graph_state[i].unsqueeze(0).to('cuda')*graph_attention[i].unsqueeze(0).to('cuda')
        with torch.no_grad():
            xi = xi.cpu()
        graph_representation.append(xi)
        progress_bar.update(1)

In [ ]:
import pickle
with open('/workspace/persistent/graph_representation_thales.pkl', 'wb') as file:
    pickle.dump(graph_representation, file)